# Handling tensors with PyTorch
At its core, PyTorch is a library that provides multidimensional arrays, called tensors in
PyTorch parlance, and an extensive library of operations on them is provided by the torch module. Both tensors and related operations can run on the CPU or GPU. The second core thing that PyTorch allows tensors to keep track of the operations performed on them and to compute derivatives of an output with respect to any of its inputs analytically via backpropagation.

**Key points**

- Numbers in Python are full-fledged objects.
- Lists in Python are meant for sequential collections of objects.
- The Python interpreter is slow compared with optimized, compiled code

[picture]

PyTorch tensors or NumPy arrays, on the other hand, are views over (typically) contiguous memory blocks containing unboxed C numeric types, not Python objects. So a 1D tensor of 1 million float numbers requires 4 million contiguous bytes to be stored, plus a small overhead for the metadata (dimensions, numeric type, and so on).



### Resources
https://openreview.net/pdf?id=BJJsrmfCZ

https://pytorch.org/docs/stable/index.html

https://pytorch.org/tutorials/

https://www.youtube.com/watch?v=f5liqUk0ZTw

In [2]:
import numpy as np
import torch
print(torch.__version__) 

1.4.0


## [Tensors](https://pytorch.org/docs/stable/tensors.html?highlight=tensor#torch.Tensor)

https://www.youtube.com/watch?v=f5liqUk0ZTw

https://pytorch.org/docs/stable/torch.html#torch.tensor

### Pytorch-Numpy interoperability

In [3]:
a_np = np.random.randn(2, 3)
print(a_np)
print() 

a = torch.tensor(a_np) # change printing precision with torch.set_printoptions()
print(a)
print()

print(a.numpy()) # it is a view on the same storage

[[ 1.17767131  0.79011993  1.0483228 ]
 [ 1.13219891 -3.00794146  1.57530639]]

tensor([[ 1.1777,  0.7901,  1.0483],
        [ 1.1322, -3.0079,  1.5753]], dtype=torch.float64)

[[ 1.17767131  0.79011993  1.0483228 ]
 [ 1.13219891 -3.00794146  1.57530639]]


In [4]:
# torch.tensor() will *construct* a tensor, so it does a copy
print(a)
a_np[0, 0] = 77
print(a)

tensor([[ 1.1777,  0.7901,  1.0483],
        [ 1.1322, -3.0079,  1.5753]], dtype=torch.float64)
tensor([[ 1.1777,  0.7901,  1.0483],
        [ 1.1322, -3.0079,  1.5753]], dtype=torch.float64)


In [5]:
# from_numpy() won't make a copy
# same with as_tensor()
a_np = np.asarray([1, 2, 3])

a = torch.from_numpy(a_np)
print(a)
a_np[0] = 44
print(a)

tensor([1, 2, 3])
tensor([44,  2,  3])


In [6]:
# how to get a python number from a tensor object?
print(a[0])

num = a[0].item()
print(num, type(num))

# Of course, we cannot get a python number from an N-dim tensor, if N>0
# a.item() # error

tensor(44)
44 <class 'int'>


In [7]:
a = torch.randint(0, 10, size=(2, 1))
b = torch.randint(0, 10, size=(2, 2))

print(a)
print(b)

tensor([[7],
        [6]])
tensor([[7, 1],
        [4, 4]])


In [8]:
print("a + b =\n{}".format(a + b))

# about ampersand operator: https://www.python.org/dev/peps/pep-0465/
print("b @ a =\n{}".format(b @ a))

# dimensions inversion
print("a.T =\n{}".format(a.T))

# matrix transpose (https://pytorch.org/docs/stable/tensors.html#torch.Tensor.t)
print("a.t() =\n{}".format(a.t()))

a + b =
tensor([[14,  8],
        [10, 10]])
b @ a =
tensor([[55],
        [52]])
a.T =
tensor([[7, 6]])
a.t() =
tensor([[7, 6]])


In [9]:
c = torch.randint(0, 10, size=(2, 3, 4, 5, 6))
print(c.size())
print(c.T.size())
print(c.transpose(0, 2).size())
# print(c.t().size()) # error

torch.Size([2, 3, 4, 5, 6])
torch.Size([6, 5, 4, 3, 2])
torch.Size([4, 3, 2, 5, 6])


In [10]:
# Different ways for creating a tensor - Factory Functions
# https://pytorch.org/cppdocs/notes/tensor_creation.html#picking-a-factory-function

print(torch.Tensor([[1, 0], [0, 1]]))
print(torch.Tensor(2, 3))

print(torch.empty(2, 3)) # it instatiate the storage without filling it

tensor([[1., 0.],
        [0., 1.]])
tensor([[1.4373e-33, 3.0623e-41, 1.4401e-33],
        [3.0623e-41, 1.3563e-19, 1.2456e-11]])
tensor([[1.4397e-33, 3.0623e-41, 1.4400e-33],
        [3.0623e-41, 1.3563e-19, 1.2456e-11]])


In [11]:
a = torch.tensor([[0, 1], [2, 467]])

print(a)
print(a.size())
print(a.dim()) # the order of the tensor
print(a.numel())
print(torch.numel(a))

tensor([[  0,   1],
        [  2, 467]])
torch.Size([2, 2])
2
4
4


In [12]:
a = torch.tensor([[0, 1], [2, 467]])

print(a.view(-1, 4))
print(a.dim())
print(a.unsqueeze(0))
print(a.squeeze(0))
print(a.fill_(33))

tensor([[  0,   1,   2, 467]])
2
tensor([[[  0,   1],
         [  2, 467]]])
tensor([[  0,   1],
        [  2, 467]])
tensor([[33, 33],
        [33, 33]])


In [13]:
a.resize_(4)
b = torch.tensor([0, 1])
dim = 0

print(a, b)
torch.cat([a, b], dim)

tensor([33, 33, 33, 33]) tensor([0, 1])


tensor([33, 33, 33, 33,  0,  1])

In [14]:
# casting
print(a.type())
print(a.double())
print(a)
# .double()
# .int()
# .byte()

torch.LongTensor
tensor([33., 33., 33., 33.], dtype=torch.float64)
tensor([33, 33, 33, 33])


In [15]:
# .cpu()
# .cuda()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Device: {}'.format(device))

Device: cpu


In [16]:
a = a.to('cuda')
b = b.to('cuda')
a + b

RuntimeError: cuda runtime error (999) : unknown error at /opt/conda/conda-bld/pytorch_1579022027550/work/aten/src/THC/THCGeneral.cpp:50

In [ ]:
# sampling from a broader range of distr (in-place): use torch.empty()
# https://pytorch.org/docs/stable/notes/serialization.html#recommend-saving-models
torch.empty(2, 3).cauchy_()

In [ ]:
# https://pytorch.org/docs/stable/torch.html#indexing-slicing-joining-mutating-ops

In [ ]:
# Saving and loading a model
# https://pytorch.org/docs/stable/torch.html#serialization